Playground for trying things out, and doing inital data exploration

In [1]:
import napari
from tifffile import imread
import numpy as np
import numpy.random
import pandas as pd
# from process_raw import humanbytes

import matplotlib.pyplot as plt
from skimage.util import img_as_uint
from skimage.util import img_as_float32
from skimage.util import img_as_ubyte

from skimage import filters
from skimage import morphology
import pickle

from skimage.draw import rectangle_perimeter
from skimage.draw import rectangle
from skimage.draw import circle_perimeter
from skimage.draw import line_aa

import time
import os
import pandas as pd

from dask_image.imread import imread as dask_imread

%gui qt

In [20]:
which_data = 'timelapse1_40_min_processed'
# which_data = 'timelapse13_onemonthlater_processed/'
# which_data = 'timelapse4_processed'
# which_data = 'timelapse5d_processed'

area = 'D01_G005'
area2 = 'G001'
# area = 'G003'
# area = 'G004'
# area = 'G005'
# area = 'G006'

inp_path2 = '/run/media/loaloa/portable_ssd/'
inp_path = '/run/media/loaloa/portable_SSD/primitives_local_processed/'

tranl_file = inp_path + f'{area}_Transmission_compr.deflate.tif'
red_file = inp_path + f'{area}_RFP_compr.deflate.tif'
green_file = inp_path + f'{area}_GFP_compr.deflate.tif'

# tranl_file2 = inp_path + f'G001_red_compr.deflate.tif'
# red_file2 = inp_path2 + f'G001_red_compr.deflate.tif'


greychannel = imread(tranl_file)
greenchannel = imread(green_file)
redchannel = imread(red_file)
greychannel = np.stack([greychannel]*len(redchannel))[:,0]
print(greychannel.shape)
print(redchannel.shape)


# redchannel = dask_imread(red_file)
# greenchannel = dask_imread(green_file)

(100, 2920, 4816)
(100, 2920, 4816)


In [8]:
with napari.gui_qt():
    viewer = napari.Viewer()

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/loaloa/programs/anaconda3/envs/bioMEA/lib/python3.8/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/loaloa/programs/anaconda3/envs/bioMEA/lib/python3.8/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/home/loaloa/programs/anaconda3/envs/bioMEA/lib/python3.8/asyncio/base_events.py", line 592, in run_until_complete
    self._check_running()
  File "/home/loaloa/programs/anaconda3/envs/bioMEA/lib/python3.8/asyncio/base_events.py", line 554, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/home/loaloa/programs/anaconda3/envs/bioMEA/li

In [21]:

if 'redchannel' in locals():
    viewer.add_image(redchannel, name='redchannel')
    viewer.layers['redchannel'].blending = 'additive'
    viewer.layers['redchannel'].colormap = 'red'
    # viewer.layers['redchannel'].contrast_limits = [0,2700]

if 'greenchannel' in locals():
    viewer.add_image(greenchannel, name='greenchannel')
    viewer.layers['greenchannel'].blending = 'additive'
    viewer.layers['greenchannel'].colormap = 'green'
    # viewer.layers['greenchannel'].contrast_limits = [0,2700]

viewer.add_image(greychannel)
viewer.layers['greychannel'].blending='additive'
# viewer.layers['greychannel'].opacity = .3
# viewer.layers['greychannel'].gamma = .6

In [10]:
from skimage.exposure import equalize_hist
motion_det = img_as_float32(equalize_hist(redchannel))

In [11]:
motion_det = np.stack([motion_det[i]-motion_det[i-1] if i else np.zeros(motion_det.shape[1:], dtype=np.float32) for i in range(motion_det.shape[0]) ])+.5
viewer.add_image(motion_det, blending='additive')

<Image layer 'motion_det [1]' at 0x7feea0147040>

In [18]:
motion_det_blur = np.stack([filters.gaussian(motion_det[t], 3) for t in range(motion_det.shape[0])])
motion_det_blur[:,~mask.astype(bool)] = 0.45
viewer.add_image(motion_det_blur, blending='additive')


In [9]:
motion_det_blur_one = motion_det_blur.copy()
motion_det_blur_one[motion_det_blur_one < .5] = .5
viewer.add_image(motion_det_blur, blending='additive')


<Image layer 'motion_det_blur [1]' at 0x7feea00194f0>

In [8]:
for i in range(1):
    axon = f'Axon_{i:0>3}'
    try:
        img = np.load(f'../outputdata/axon_distances/{axon}_distance_img.npy')
        print('Found', axon)
    except FileNotFoundError:
        print('Didnt find ', axon)
        continue
    viewer.add_image(img, name=axon, colormap='fire')
    del img

    if i >20:
        break
    

Didnt find  Axon_000
